## imports

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

import re
import time

In [2]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

In [3]:
parent_url = 'https://auto.ru/cars/kia/all/'

## reading test data

In [4]:
test_data = pd.read_pickle("data/test.pkl.zip", compression="zip")

In [5]:
test_data.iloc[12561]["description"]

'Машина в отличном состоянии!!17.10.2020 было сделано Т.О заменено масло в коробке передач в двигателе,так же заменены все фильтра.Почти максимальная комплектация кроме люка!!! Без ключевой доступ. В ДТП не была не билась.Красился капот и передний бампер так как от времени были сколы.Капот фары туманки затянуты броне пленкой.Делали для себя супруга не ездит вся причина продажи.. Новая летняя резина. Продаю не спеша. '

In [6]:
test_data.iloc[1550].to_frame()

,1550
bodyType,внедорожник 5 дв.
brand,SKODA
car_url,https://auto.ru/cars/used/sale/skoda/kodiaq/11...
color,серый
complectation_dict,"{""id"":""21404914"",""name"":""Hockey Edition"",""avai..."
description,Куплена в январе 2019 г. у официала. Производс...
engineDisplacement,2.0 LTR
enginePower,180 N12
equipment_dict,"{""cruise-control"":true,""tinted-glass"":true,""es..."
fuelType,бензин


## setup chrome driver

In [7]:
!rm *chromedriver*
!wget https://chromedriver.storage.googleapis.com/99.0.4844.51/chromedriver_linux64.zip
!unzip chromedriver_linux64.zip
!rm chromedriver_linux64.zip
!ls

--2022-03-19 01:07:48--  https://chromedriver.storage.googleapis.com/99.0.4844.51/chromedriver_linux64.zip
Распознаётся chromedriver.storage.googleapis.com (chromedriver.storage.googleapis.com)… 108.177.14.128, 2a00:1450:4010:c0f::80
Подключение к chromedriver.storage.googleapis.com (chromedriver.storage.googleapis.com)|108.177.14.128|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 200 OK
Длина: 7009102 (6,7M) [application/zip]
Сохранение в: «chromedriver_linux64.zip»

chromedriver_linux6 100%[===================>]   6,68M  9,03MB/s    за 0,7s    

2022-03-19 01:07:48 (9,03 MB/s) - «chromedriver_linux64.zip» сохранён [7009102/7009102]

Archive:  chromedriver_linux64.zip
  inflating: chromedriver            
chromedriver  geckodriver.log  README.md
data	      LICENSE	       UsedCars_Project_Module_6_parser.ipynb


In [8]:
options = webdriver.ChromeOptions()
# options.add_argument('headless')
driver = webdriver.Chrome("./chromedriver", options=options)

/tmp/ipykernel_21473/635207263.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("./chromedriver", options=options)


## getting list of links

In [9]:
models = test_data["brand"].apply(lambda x: x.lower()).unique().tolist()
models

['skoda',
 'audi',
 'honda',
 'volvo',
 'bmw',
 'nissan',
 'infiniti',
 'mercedes',
 'toyota',
 'lexus',
 'volkswagen',
 'mitsubishi']

In [10]:
urls_list = []

for model in models:
    
    driver.get(f"https://auto.ru/cars/{model}/all/")
    html = driver.find_elements(By.CLASS_NAME, 'Link ListingItemTitle__link')
    rr = driver.find_elements(By.CLASS_NAME, 'ListingItemTitle__link')
    
    for item in rr:
        temp = item.get_attribute('href')

        # getting links only from the first page - to add all pages

        if 'used' in temp:
            urls_list.append(temp)


In [11]:
urls_list

['https://auto.ru/cars/used/sale/skoda/rapid/1115076168-8671a836/',
 'https://auto.ru/cars/used/sale/skoda/octavia/1114710377-b758a05a/',
 'https://auto.ru/cars/used/sale/skoda/rapid/1114913712-f06b2734/',
 'https://auto.ru/cars/used/sale/skoda/superb/1115084971-8f113dc7/',
 'https://auto.ru/cars/used/sale/skoda/rapid/1105113801-81a08770/',
 'https://auto.ru/cars/used/sale/skoda/octavia/1105858333-f74fed4c/',
 'https://auto.ru/cars/used/sale/skoda/octavia/1114998346-3887ee97/',
 'https://auto.ru/cars/used/sale/skoda/rapid/1104942965-f8b66dcb/',
 'https://auto.ru/cars/used/sale/skoda/octavia/1115048315-0827fa9f/',
 'https://auto.ru/cars/used/sale/skoda/karoq/1115010334-49b78e39/',
 'https://auto.ru/cars/used/sale/skoda/superb/1114963102-13171e22/',
 'https://auto.ru/cars/used/sale/skoda/octavia_rs/1115009109-003baad4/',
 'https://auto.ru/cars/used/sale/skoda/octavia/1114786645-fd032984/',
 'https://auto.ru/cars/used/sale/skoda/octavia/1114671743-8ea4534e/',
 'https://auto.ru/cars/used/s

In [12]:
len(urls_list)

374

In [13]:
driver.get('https://auto.ru/cars/used/sale/kia/cerato/1114739047-04c7e4fe/')

In [14]:
def parse_auto_ru(url):
        
    driver.get(url)
    
    year = int(driver.find_element(By.CLASS_NAME, 'CardInfoRow_year').text.split('\n')[1])
    bodytype = driver.find_element(By.CLASS_NAME, 'CardInfoRow_bodytype').text.split('\n')[1]
    kmage = int(re.sub('\D', '', driver.find_element(By.CLASS_NAME, 'CardInfoRow_kmAge').text))
    color = driver.find_element(By.CLASS_NAME, 'CardInfoRow_color').text.split('\n')[1]
    engine = driver.find_element(By.CLASS_NAME, 'CardInfoRow_engine').text.split('/')
    engineDisplacement = float(re.findall('(\d+.\d+)', engine[0])[0])
    enginePower = int(re.findall('\d+', engine[1])[0])
    fuelType = engine[2]
    vehicleTransmission = driver.find_element(By.CLASS_NAME, 'CardInfoRow_transmission').text.split('\n')[1]
    drive = driver.find_element(By.CLASS_NAME, 'CardInfoRow_drive').text.split('\n')[1]
    wheel = driver.find_element(By.CLASS_NAME, 'CardInfoRow_wheel').text.split('\n')[1]
    state = driver.find_element(By.CLASS_NAME, 'CardInfoRow_state').text.split('\n')[1]
    owner = driver.find_element(By.CLASS_NAME, 'CardInfoRow_ownersCount').text.split('\n')[1]
    pts = driver.find_element(By.CLASS_NAME, 'CardInfoRow_pts').text.split('\n')[1]
    customs = driver.find_element(By.CLASS_NAME, 'CardInfoRow_customs').text.split('\n')[1]
    try:
        owningTime = driver.find_element(By.CLASS_NAME, 'CardInfoRow_owningTime').text.split('\n')[1]
    except NoSuchElementException:
        owningTime = None
    try:
        description = driver.find_element(By.CLASS_NAME, 'CardDescriptionHTML').text
    except NoSuchElementException:
        description = None
    sell_id = int(re.findall('\d+', driver.find_element(By.CLASS_NAME, 'CardHead__id').text)[0])
    
    data_dict = {
        'year': year, 'bodytype': bodytype, 'kmage': kmage, 'color': color, 
        'engineDisplacement': engineDisplacement, 'enginePower': enginePower, 
        'fuelType': fuelType, 'vehicleTransmission': vehicleTransmission, 'drive': drive, 
        'wheel': wheel, 'state': state, 'owner': owner, 'pts': pts, 'customs': customs, 
        'owningTime': owningTime, 'description': description, 'sell_id': sell_id
    }
    
    return data_dict

## reading data from urls

In [29]:
valid_data = pd.DataFrame(
    columns=[
        "year",
        "bodytype",
        "kmage",
        "color",
        "engineDisplacement",
        "enginePower",
        "fuelType",
        "vehicleTransmission",
        "drive",
        "wheel",
        "state",
        "owner",
        "pts",
        "customs",
        "owningTime",
        "description",
        "sell_id",
    ]
)


In [30]:
for i, url in enumerate(urls_list):
    print(f"processing url {i + 1} of {len(urls_list)}: {url} ...", end="")
    try:
        new_row = parse_auto_ru(url)
        valid_data = pd.concat(
            [valid_data, pd.DataFrame(new_row, index=[0])], ignore_index=True
        )
        print("done")
    except NoSuchElementException as noelem:
        print(noelem)


processing url 1 of 374: https://auto.ru/cars/used/sale/skoda/rapid/1115076168-8671a836/ ...done
processing url 2 of 374: https://auto.ru/cars/used/sale/skoda/octavia/1114710377-b758a05a/ ...done
processing url 3 of 374: https://auto.ru/cars/used/sale/skoda/rapid/1114913712-f06b2734/ ...done
processing url 4 of 374: https://auto.ru/cars/used/sale/skoda/superb/1115084971-8f113dc7/ ...done
processing url 5 of 374: https://auto.ru/cars/used/sale/skoda/rapid/1105113801-81a08770/ ...done
processing url 6 of 374: https://auto.ru/cars/used/sale/skoda/octavia/1105858333-f74fed4c/ ...done
processing url 7 of 374: https://auto.ru/cars/used/sale/skoda/octavia/1114998346-3887ee97/ ...done
processing url 8 of 374: https://auto.ru/cars/used/sale/skoda/rapid/1104942965-f8b66dcb/ ...done
processing url 9 of 374: https://auto.ru/cars/used/sale/skoda/octavia/1115048315-0827fa9f/ ...done
processing url 10 of 374: https://auto.ru/cars/used/sale/skoda/karoq/1115010334-49b78e39/ ...done
processing url 11 of

In [31]:
valid_data

,year,bodytype,kmage,color,engineDisplacement,enginePower,fuelType,vehicleTransmission,drive,wheel,state,owner,pts,customs,owningTime,description,sell_id
0,2015,лифтбек,149509,серый,1.6,105,Бензин,автоматическая,передний,Левый,Не требует ремонта,3 или более,Дубликат,Растаможен,None,Усилитель руля!!! Кондиционер!!! Парктроник за...,1115076168
1,2017,лифтбек,98000,серый,1.8,180,Бензин,роботизированная,передний,Левый,Не требует ремонта,2 владельца,Оригинал,Растаможен,None,Покупка автомобиля в кредит возможна по старым...,1114710377
2,2015,лифтбек,189948,белый,1.2,75,Бензин,механическая,передний,Левый,Не требует ремонта,2 владельца,Оригинал,Растаможен,None,24 и 25 марта большая распродажа автомобилей с...,1114913712
3,2019,лифтбек,25000,чёрный,1.8,180,Бензин,роботизированная,передний,Левый,Не требует ремонта,2 владельца,Оригинал,Растаможен,11 месяцев,Новый автомобиль в топовой комплектации.\nБез ...,1115084971
4,2014,лифтбек,89000,серебристый,1.6,105,Бензин,автоматическая,передний,Левый,Не требует ремонта,2 владельца,Оригинал,Растаможен,None,Продаю технически исправный автомобиль любые п...,1105113801
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
361,2018,внедорожник 5 дв.,86844,серебристый,3.0,227,Бензин,автоматическая,полный,Левый,Не требует ремонта,1 владелец,Оригинал,Растаможен,None,⭐ Более 150 автомобилей на сайте used.autograd...,1106595239
362,2018,внедорожник 5 дв.,83498,коричневый,3.0,209,Бензин,автоматическая,полный,Левый,Не требует ремонта,1 владелец,Оригинал,Растаможен,None,.\nТолько до 31 марта мы ВОЗВРАЩАЕМ ВЫГОДУ:\n✔...,1106566926
363,2013,внедорожник 5 дв.,112000,белый,2.4,167,Бензин,вариатор,полный,Левый,Не требует ремонта,3 или более,Оригинал,Растаможен,None,▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬\n\n-Льготные пр...,1115050646
364,2021,внедорожник 5 дв.,16872,чёрный,2.4,181,Дизель,автоматическая,полный,Левый,Не требует ремонта,1 владелец,Оригинал,Растаможен,None,"В продаже автомобили от компании АО ABTODOM, о...",1114610546


In [32]:
valid_data.to_pickle("data/parsed_first_pages.pkl.zip", compression="zip")